### Using the OpenAI Library to Programmatically Access GPT-3.5-turbo!

In [2]:
%pip install openai cohere tiktoken -q 

Note: you may need to restart the kernel to use updated packages.


In order to get started, we'll need to provide our OpenAI API Key - detailed instructions can be found [here](https://github.com/AI-Maker-Space/Interactive-Dev-Environment-for-LLM-Development#-setting-up-keys-and-tokens)!

In [3]:
import os
import openai
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Please enter your OpenAI API key")
openai.api_key = os.environ["OPENAI_API_KEY"]

### Our First Prompt

You can reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) if you get stuck!

Let's create a `ChatCompletion` model to kick things off!

There are three "roles" available to use:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-chatgpt-api-transition-guide)

Let's just stick to the `user` role for now and send our first message to the endpoint!

If we check the documentation, we'll see that it expects it in a list of prompt objects - so we'll be sure to do that!

In [9]:
from openai import OpenAI

client = OpenAI()

In [12]:
YOUR_PROMPT = "What is the difference between LangChain and LlamaIndex?"

client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role":"user", "content":YOUR_PROMPT}
    ]
)

ChatCompletion(id='chatcmpl-9LFTUDSgOINy3a1NDLTG9kD5S5R4Y', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='LangChain and LlamaIndex are two different projects in the cryptocurrency and blockchain space.\n\n1. LangChain: LangChain is a project that focuses on solving the challenge of language barriers in the blockchain and cryptocurrency industry. The platform aims to provide language translation services for various blockchain projects, making them more accessible to a global audience. LangChain utilizes blockchain technology to ensure the accuracy and security of translations, and it also offers additional services such as language tutoring and community support.\n\n2. LlamaIndex: LlamaIndex, on the other hand, is a platform that provides users with real-time market data and analysis for various cryptocurrencies and blockchain projects. It offers insights, charts, and statistics to help users make informed investment decisions in t

As you can see, the prompt comes back with a tonne of information that we can use when we're building our applications!

We'll be building some helper functions to pretty-print the returned prompts and to wrap our messages to avoid a few extra characters of code!

### Helper Functions

In [19]:
from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: str, model:str = "gpt-3.5-turbo") -> dict:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def user_prompt(message: str) -> dict: 
    return {"role": "user", "content": message}

def system_prompt(message: str) -> dict: 
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict: 
    return {"role": "assistant", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

### Testing Helper Functions

In [23]:
LANG_LLAMA_PROMPT = "What is the difference between LangChain and LlamaIndex?"
EARTH_SUN_PROMPT = "What is the distance between earth and sun in miles?"
BLACK_HOLE_PROMPT = "What is the closest black hole to us?"
STOCK_PROMPT = "What is a cyber security stock that is valued less than it's growth potential?"

messages = [user_prompt(STOCK_PROMPT)]

chatgpt_response = get_response(client, messages)

pretty_print(chatgpt_response)

One cyber security stock that fits this description is CrowdStrike Holdings Inc. (CRWD). CrowdStrike is a fast-growing company that provides cloud-delivered endpoint protection and threat intelligence services. The company's stock price has risen significantly since its IPO in 2019, but many analysts believe that it still has significant growth potential in the rapidly expanding cyber security market. With a market capitalization of over $50 billion and a leadership position in cloud-based security solutions, CrowdStrike is considered a top pick for investors looking to capitalize on the growing demand for cyber security services.

Let's focus on extending this a bit, and incorporate a `system` message as well!

Again, the API expects our prompts to be in a list - so we'll be sure to set up a list of prompts!

>REMINDER: The system message acts like an overarching instruction that is applied to your user prompt. It is appropriate to put things like general instructions, tone/voice suggestions, and other similar prompts into the system prompt.

In [30]:
list_of_prompts = [
    system_prompt("You are irate and extremely hungry."),
    user_prompt("Do you prefer cubed ice or crushed ice?")
]
chatgpt_response = get_response(client, list_of_prompts)
pretty_print(chatgpt_response)

I don't have a preference for ice right now because I am absolutely starving and need food ASAP!

Let's try that same prompt again, but modify only our system prompt!

In [31]:
list_of_prompts = [
    system_prompt("You are joyful and having an awesome day!"),
    user_prompt("Do you prefer cubed ice or crushed ice?")
]

chatgpt_response = get_response(client, list_of_prompts)
pretty_print(chatgpt_response)

I don't have a preference, but I can imagine enjoying the sound of cubed ice clinking in a glass on a warm summer day. How about you?

While we're only printing the responses, remember that OpenAI is returning the full payload that we can examine and unpack!

In [34]:
print(chatgpt_response)

ChatCompletion(id='chatcmpl-9LHDPm0Na9ISXnY06nsiAclRgtkto', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I don't have a preference, but I can imagine enjoying the sound of cubed ice clinking in a glass on a warm summer day. How about you?", role='assistant', function_call=None, tool_calls=None))], created=1714858091, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_3b956da36b', usage=CompletionUsage(completion_tokens=33, prompt_tokens=30, total_tokens=63))


### Few-Shot Prompting

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-3.5-turbo` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [35]:
list_of_prompts = [
    user_prompt("Please use the words 'Stumple' and 'Fallbean' in a sentence")
]

simple_response = get_response(client, list_of_prompts)
pretty_print(simple_response)

I picked a Stimple leaf from the Fallbean plant in the garden.